Author: Yongquan Xie, Nathaniel Blair-Stahn<br>
Date: July 25, 2019<br>
Purpose: SQ-LNS presentation Nigeria results preparation<br>
Note: Yongquan and Nathaniel will give this presentation on August 1, 2019

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

pd.set_option('display.max_rows', 8)

!date
!whoami

## Load output data and aggregate over random seeds

In [ ]:
cause_names = ['lower_respiratory_infections', 'measles', 'diarrheal_diseases', 
               'protein_energy_malnutrition', 'iron_deficiency', 'other_causes']
risk_names = ['anemia', 'child_stunting', 'child_wasting']

template_cols = ['coverage', 'duration', 'child_stunting_permanent', 
                 'child_wasting_permanent', 'iron_deficiency_permanent', 
                 'iron_deficiency_mean', 'cause', 'measure', 'input_draw']

result_dir = '/share/costeffectiveness/results/sqlns/presentation/nigeria/2019_07_23_10_57_25'

In [ ]:
# note that we have applied coefficient of variation as constant with different sqlns effect on iron deficiency
def clean_and_aggregate(path, filename):
#     r = pd.read_hdf(path + 'nigeria/2019_07_18_13_20_17/output.hdf')
    r = pd.read_hdf(f'{path}/{filename}')
    r.rename(columns={'sqlns.effect_on_child_stunting.permanent': 'child_stunting_permanent',
                      'sqlns.effect_on_child_wasting.permanent': 'child_wasting_permanent',
                      'sqlns.effect_on_iron_deficiency.permanent': 'iron_deficiency_permanent',
                      'sqlns.effect_on_iron_deficiency.mean': 'iron_deficiency_mean',
                      'sqlns.program_coverage': 'coverage',
                      'sqlns.duration': 'duration'}, inplace=True)
    r['coverage'] *= 100
    # The 'sqlns_treated_days' column got subtracted in the wrong order for the 2019_07_23_10_57_25 run:
    r['sqlns_treated_days'] = -1 * r['sqlns_treated_days'] # This line should be deleted once the code is fixed
    r = r.groupby(['coverage', 'duration', 'child_stunting_permanent', 'child_wasting_permanent', 'iron_deficiency_permanent', 'iron_deficiency_mean', 'input_draw']).sum()
    return r

In [ ]:
# Load outpt data - as of 2019-07-25 there are random seeds missing
r = clean_and_aggregate(result_dir, 'output.hdf')
# Raw data aggregated by random seed, with intervention columns renamed
r

## Get a list of the unique draws for plotting by draw

In [ ]:
draws = r.reset_index().input_draw.unique()
draws

## Plot total YLLs and YLDs vs. coverage for each draw

Raw YLLs and YLDs are plotted side by side with the rates per 100,000 person years. Plots should be monotonically decreasing as coverage level increases.

Create a `pandas.IndexSlice` object to easily select with the multi-index of the original aggregated dataframe.

In [ ]:
# Create a pandas IndexSlice object to easily multi-index the original dataframe
idx = pd.IndexSlice
r.loc[idx[:, 365.25, False, False, False, 0.895, 55],
      ['years_of_life_lost', 'years_lived_with_disability', 'person_time']].reset_index()

In [ ]:
@interact()
def plot_total_dalys_by_draw(duration=[365.25, 730.50],
                    cgf_permanent=[False, True],
                    iron_permanent=[False, True],
                    iron_mean=[0.895, 4.475, 8.950],
                    input_draw = draws,
                  ):
    
    data = r.loc[idx[:, duration, cgf_permanent, cgf_permanent, iron_permanent, iron_mean, input_draw],
      ['years_of_life_lost', 'years_lived_with_disability', 'person_time']].reset_index()
    
    fig, ax = plt.subplots(2,2, figsize=(12,8))
    
    xx = data['coverage']
    
    measures_short_names = {'years_of_life_lost': 'YLL', 'years_lived_with_disability': 'YLD'}

    for i, (measure, short_name) in enumerate(measures_short_names.items()):
        ax[i,0].plot(xx, data[measure], '-o')
        ax[i,1].plot(xx, 100_000*data[measure] / data['person_time'], '-o', color='orange')
    
        ax[i,0].set_title(f'Total {short_name} count vs. coverage', fontsize=20)
        ax[i,0].set_xlabel('Program Coverage (%)', fontsize=16)
        ax[i,0].set_ylabel(f'{short_name}s', fontsize=20)
        ax[i,0].grid()
#         ax[i,0].legend(loc=(0.8, -.25), fontsize=14)

        ax[i,1].set_title(f'Total {short_name} rate vs. coverage', fontsize=20)
        ax[i,1].set_xlabel('Program Coverage (%)', fontsize=16)
        ax[i,1].set_ylabel(f'{short_name}s per 100,000 person years', fontsize=12)
        ax[i,1].grid()
        
    fig.tight_layout()

## Plot treated days and estimated fraction of population treated for all draws

In [ ]:
r.filter(regex='treated_days|population|person_time')

In [ ]:
# The fraction of population tracked is about 54.4% for all scenarios and draws.
# Why? How do you compute this?
(r['total_population_tracked']/r['total_population']).describe()

In [ ]:
days_per_year = 365.25
years_of_simulation = 5

@interact()
def plot_treated_days_by_draw(duration=[365.25, 730.50],
                    cgf_permanent=[False, True],
                    iron_permanent=[False, True],
                    iron_mean=[0.895, 4.475, 8.950],
                    input_draw = draws,
                  ):
    
    data = r.loc[idx[:, duration, cgf_permanent, cgf_permanent, iron_permanent, iron_mean, input_draw],
      ['sqlns_treated_days', 'total_population_living', 'total_population_tracked', 'person_time']].reset_index()
    
    fig, ax = plt.subplots(1,2, figsize=(13,6))
    
    xx = data['coverage']
    

    ax[0].plot(xx, data['sqlns_treated_days'] / days_per_year, '-o')
#     # This is computing something like "average person years per treatment year for a treated simulant",
#     # then multiplying that by the number of treated years over the number of person years.
#     ax[1].plot(xx,
#                (data['total_population_living'] / data['total_population_tracked']) *
#                years_of_simulation * data['sqlns_treated_days'] / (duration * data['person_time']),
#                '-o', color='orange')
    ax[1].plot(xx, data['sqlns_treated_days'] / (duration * data['total_population_tracked']),
               '-o', color='orange')

    ax[0].set_title('Treated years vs. coverage', fontsize=20)
    ax[0].set_xlabel('Program Coverage (%)', fontsize=16)
    ax[0].set_ylabel('SQ-LNS treated years', fontsize=20)
    ax[0].grid()
#         ax[i,0].legend(loc=(0.8, -.25), fontsize=14)

    ax[1].set_title('Estimated fraction of\npopulation treated vs. coverage', fontsize=20)
    ax[1].set_xlabel('Program Coverage (%)', fontsize=16)
#     ax[1].set_ylabel('(survival-rate)\nx (simulation-duration / treatment-duration)\nx (treated-years / person-years)', fontsize=12)
    ax[1].set_ylabel('sqlns_treated_time /\n(treatment_duration x population_tracked)', fontsize=12)
    ax[1].grid()
        
    fig.tight_layout()

### Based on the graphs, estimated coverage is close to program coverage -- how close?

The maximum difference is less than 2%, with a mean around 0.28%.

In [ ]:
(100*r.sqlns_treated_days / 
 (r.index.get_level_values('duration') * r.total_population_tracked)
 - r.index.get_level_values('coverage')).describe()

### If we invert the equation to estimate treated years from coverage, how close do we get?

The maximum difference is about 1764 treated years, with a mean around 208 treated years.

In [ ]:
((r.sqlns_treated_days
 - 0.01*r.index.get_level_values('coverage')
 * r.index.get_level_values('duration')
 * r.total_population_tracked)/days_per_year).describe()

## Define functions to transform data into "long" form suitible for more analysis/graphing

In [ ]:
def standardize_shape(data, measure):
    measure_data = data.loc[:, [c for c in data.columns if measure in c]]
    measure_data = measure_data.stack().reset_index().rename(columns={'level_7': 'label', 0: 'value'})
    if 'due_to' in measure:
        measure, cause = measure.split('_due_to_', 1)
        measure_data.loc[:, 'measure'] = measure
        measure_data.loc[:, 'cause'] = cause
    else:
        measure_data.loc[:, 'measure'] = measure  
    measure_data.drop(columns='label', inplace=True)
    
    return measure_data

In [ ]:
def get_person_time(data):
    pt = standardize_shape(data, 'person_time')
    pt = pt.rename(columns={'value': 'person_time'}).drop(columns='measure')
    return pt

In [ ]:
def get_treated_days(data):
    treated = standardize_shape(data, 'sqlns_treated_days')
    treated = treated.rename(columns={'value': 'sqlns_treated_days'}).drop(columns='measure')
    return treated

In [ ]:
get_person_time(r)

In [ ]:
get_treated_days(r)

In [ ]:
def get_disaggregated_results(data, cause_names):
    deaths = []
    ylls = []
    ylds = []
    dalys = []
    for cause in cause_names:
        if cause in cause_names[:4]:
            deaths.append(standardize_shape(data, f'death_due_to_{cause}'))
            
            ylls_sub = standardize_shape(data, f'ylls_due_to_{cause}')
            ylds_sub = standardize_shape(data, f'ylds_due_to_{cause}')
            dalys_sub = (ylds_sub.set_index([c for c in template_cols if c != 'measure']) + \
                         ylls_sub.set_index([c for c in template_cols if c != 'measure'])).reset_index()
            dalys_sub['measure'] = 'dalys'
            
            ylls.append(ylls_sub)
            ylds.append(ylds_sub)
            dalys.append(dalys_sub)
        elif cause == 'iron_deficiency':
            ylds_sub = standardize_shape(data, f'ylds_due_to_{cause}')     
            dalys_sub = ylds_sub.copy()
            dalys_sub['measure'] = 'dalys'
            
            ylds.append(ylds_sub)
            dalys.append(dalys_sub)
        else: # cause == 'other_causes'
            deaths.append(standardize_shape(data, f'death_due_to_{cause}'))
            
            ylls_sub = standardize_shape(data, f'ylls_due_to_{cause}')
            dalys_sub = ylls_sub.copy()
            dalys_sub['measure'] = 'dalys'
            
            ylls.append(ylls_sub)
            dalys.append(dalys_sub)
    
    death_data = pd.concat(deaths, sort=False)
    yll_data = pd.concat(ylls, sort=False)
    yld_data = pd.concat(ylds, sort=False)
    daly_data = pd.concat(dalys, sort=False)
    
    output = pd.concat([death_data, yll_data, yld_data, daly_data], sort=False)
    output = output.set_index(template_cols).sort_index()
    
    return output.reset_index()

## Get the transformed data

In [ ]:
output = get_disaggregated_results(r, cause_names)

In [ ]:
output

## Add columns recording person time and treated time for each (scenario, draw, cause) combination

In [ ]:
join_columns = [c for c in template_cols if c not in ['cause', 'measure']]
df = output.merge(get_person_time(r), on=join_columns).merge(get_treated_days(r), on=join_columns)
df

## Function to plot mortality/DALY/YLL/YLD by disease at the draw level

Each raw measure is plotted side by side with its rate per 100,000 person years. Plots should be monotonically decreasing as coverage level increases.

In [ ]:
@interact()
def plot_cause_spceific_dalys_by_draw(duration=[365.25, 730.50],
                    cgf_permanent=[False, True],
                    iron_permanent=[False, True],
                    iron_mean=[0.895, 4.475, 8.950],
                    input_draw = df.input_draw.unique(),
                    measure = df.measure.unique(),
                    include_other_causes=True,
                  ):
    
    data = df.loc[(df.duration == duration)
                  & (df.child_stunting_permanent == cgf_permanent)
                  & (df.child_wasting_permanent == cgf_permanent)
                  & (df.iron_deficiency_permanent == iron_permanent)
                  & (df.iron_deficiency_mean == iron_mean)
                  & (df.input_draw == input_draw)
                  & (df.measure == measure)]
    
    fig, ax = plt.subplots(1,2, figsize=(18,8))
    
    # 'other_causes' value is much higher - can omit by indexing with [:-1]
    displayed_causes = cause_names if include_other_causes else cause_names[:-1]
    for cause in displayed_causes:
        data_sub = data.loc[data.cause == cause]
        
        xx = data_sub['coverage']
        value = data_sub['value']
        value_over_pt = 100_000* data_sub['value'] / data_sub['person_time']
        
        ax[0].plot(xx, value, '-o', label=cause)
        ax[1].plot(xx, value_over_pt, '-o')
        
    singular_measure = measure if measure=='death' else measure[:-1]
    plural_measure = 'deaths' if measure=='death' else measure
    
    ax[0].set_title(f'{singular_measure.upper()} count by disease vs. coverage', fontsize=20)
    ax[0].set_xlabel('Program Coverage (%)', fontsize=20)
    ax[0].set_ylabel(f'{plural_measure.upper()}', fontsize=20)
    ax[0].grid()
    ax[0].legend(loc=(0.9, -.3))
    
    ax[1].set_title(f'{singular_measure.upper()} rate by disease vs. coverage', fontsize=20)
    ax[1].set_xlabel('Program Coverage (%)', fontsize=20)
    ax[1].set_ylabel(f'{plural_measure.upper()} per 100,000 person years', fontsize=20)
    ax[1].grid()

## Calculate averted DALYs

In [ ]:
def get_averted_results(df):
    bau = df[df.coverage == 0.0].drop(columns=['coverage', 'person_time'])
    t = pd.merge(df, bau, on=template_cols[1:], suffixes=['', '_bau'])
    t['averted'] = t['value_bau'] - t['value']
#     t.drop(columns='value_bau', inplace=True)
    
    t['value'] = (t['value']/t['person_time']) * 100_000
    t['averted'] = (t['averted']/t['person_time']) * 100_000
    
    return t

In [ ]:
get_averted_results(df)

In [ ]:
def get_final_table(data):
    g = data.groupby(template_cols[:-1])[['person_time', 'value', 'averted']]\
            .describe(percentiles=[.025, .975])
    
    table = g.filter([('value', 'mean'), ('value', '2.5%'), ('value', '97.5%'),
                      ('person_time', 'mean'), ('person_time', '2.5%'), ('person_time', '97.5%'),
                      ('averted', 'mean'), ('averted', '2.5%'), ('averted', '97.5%')])
    return table

In [ ]:
table_shell = get_final_table(get_averted_results(df))

In [ ]:
table_shell

## Original plot of averted DALYs that tipped us off that something was wrong

The graph is not monotone with coverage.

In [ ]:
@interact()
def plot_dalys_averted(duration=[365.25, 730.50],
                       cgf_permanent=[False, True],
                       iron_permanent=[False, True],
                       iron_mean=[0.895, 4.475, 8.950],
                       include_other_causes=False
                      ):
    
    df = table_shell.reset_index()
    
    data = df.loc[(df.duration == duration)
                  & (df.child_stunting_permanent == cgf_permanent)
                  & (df.child_wasting_permanent == cgf_permanent)
                  & (df.iron_deficiency_permanent == iron_permanent)
                  & (df.iron_deficiency_mean == iron_mean)
                  & (df.measure == 'dalys')]
    
    plt.figure(figsize=(12, 8))
    
    # 'other_causes' value is much higher - can omit by indexing with [:-1]
    displayed_causes = cause_names if include_other_causes else cause_names[:-1]
    for cause in displayed_causes:
        data_sub = data.loc[data.cause == cause]
        
        xx = data_sub['coverage']
        mean = data_sub[('averted', 'mean')]
        lb = data_sub[('averted', '2.5%')]
        ub = data_sub[('averted', '97.5%')]
        
        plt.plot(xx, mean, '-o', label=cause)
        plt.fill_between(xx, lb, ub, alpha=0.1)
    
    plt.title('Nigeria')
    plt.xlabel('Program Coverage (%)')
    plt.ylabel('DALYs Averted (per100,000PY)')
    plt.legend(loc=(1.05, .05))
    plt.grid()

## Check averted DALYs data to find the problem

The problem with the averted DALYs graphs is that DALYs are being subtracted **before** dividing by person time, whereas the subtraction should actually happen **after** dividing by the person time for the specific scenario:
```python
def get_averted_results(df):
    bau = df[df.coverage == 0.0].drop(columns=['coverage', 'person_time'])
    t = pd.merge(df, bau, on=template_cols[1:], suffixes=['', '_bau'])
    # NOTE: This computes the difference in DALY count, not DALY rate:
    t['averted'] = t['value_bau'] - t['value']
#     t.drop(columns='value_bau', inplace=True)
    
    t['value'] = (t['value']/t['person_time']) * 100_000
    # NOTE: This calculation ignores person time for 0% coverage, dividing
    #  the difference in DALYs by the person time for the intervention:
    t['averted'] = (t['averted']/t['person_time']) * 100_000
    
    return t
```

Dividing by person time first would ensure that we're subtracting quantities that are on comparable scales.

In [ ]:
averted_df = get_averted_results(df)
averted_df

In [ ]:
# r.loc[idx[:, 365.25, False, False, False, 0.895, 357],
#       ['death_due_to_other_causes', 'death_due_to_diarrheal_diseases', 'ylds_due_to_iron_deficiency']]
sub_df = averted_df.loc[(averted_df.duration==365.25)
               & (averted_df.child_stunting_permanent==False)
               & (averted_df.child_wasting_permanent==False)
               & (averted_df.iron_deficiency_permanent==False)
               & (averted_df.iron_deficiency_mean==0.895)
               & (averted_df.input_draw==357)
               & (averted_df.cause=='diarrheal_diseases')
               & (averted_df.measure=='dalys')
               ,['coverage', 'person_time', 'value', 'value_bau', 'averted']
              ]
sub_df

In [ ]:
# This should be 0, based on calculation in `get_averted_results()`.
sub_df['value_bau'] - (sub_df['value']+sub_df['averted'])*sub_df['person_time']/100_000

In [ ]:
# This computes the original DALY values for each coverage level.
# We can verify the numbers against the above graph of DALYs by disease for draw 357.
sub_df['value']*sub_df['person_time']/100_000

In [ ]:
# This computes the original difference in DALYs between X% coverage and 0% coverage.
# Note the very large value at 40%, matching the above graphs.
sub_df['averted']*sub_df['person_time']/100_000

## Draw a graph of aggregated DALYs per 100,000 PY, rather than averted DALYs

The actual averted DALYs per $10^5$ PY would be the difference in y values of this graph from 0% to the various coverage levels. However, to draw the new graph for averted DALYs, we need to rewrite the `get_averted_dalys()` function and re-aggregate the data; see below.

In [ ]:
@interact()
def plot_dalys_per_1e5_py(duration=[365.25, 730.50],
                       cgf_permanent=[False, True],
                       iron_permanent=[False, True],
                       iron_mean=[0.895, 4.475, 8.950],
                        include_other_causes=False):
    
    df = table_shell.reset_index()
    
    data = df.loc[(df.duration == duration)
                  & (df.child_stunting_permanent == cgf_permanent)
                  & (df.child_wasting_permanent == cgf_permanent)
                  & (df.iron_deficiency_permanent == iron_permanent)
                  & (df.iron_deficiency_mean == iron_mean)
                  & (df.measure == 'dalys')]
    
    plt.figure(figsize=(12, 8))
    
    # 'other_causes' value is much higher - can omit by indexing with [:-1]
    displayed_causes = cause_names if include_other_causes else cause_names[:-1]
    for cause in displayed_causes:
        data_sub = data.loc[data.cause == cause]
        
        xx = data_sub['coverage']
        mean_per_py = data_sub[('value', 'mean')]
        lb = data_sub[('value', '2.5%')]
        ub = data_sub[('value', '97.5%')]
        
        plt.plot(xx, mean_per_py, '-o', label=cause)
        plt.fill_between(xx, lb, ub, alpha=0.1)
    
    plt.title('Nigeria')
    plt.xlabel('Program Coverage (%)')
    plt.ylabel('DALYs per 100,000 PY')
    plt.legend(loc=(1.05, .05))
    plt.grid()

## Rewrite averted DALYs function to compute in rate space, and draw new graphs

In [ ]:
def get_averted_results_in_rate_space(df):
    bau = df[df.coverage == 0.0].drop(columns=['coverage'])
    t = pd.merge(df, bau, on=template_cols[1:], suffixes=['', '_bau'])
    t['averted'] = (t['value_bau']/t['person_time_bau'] - t['value']/t['person_time']) * 100_000
    
#     t['value'] = (t['value']/t['person_time']) * 100_000
#     t['averted'] = (t['averted']/t['person_time']) * 100_000
    
    return t

In [ ]:
averted_rate_space_df = get_averted_results_in_rate_space(df)
averted_rate_space_df

In [ ]:
aggregated_results_df = get_final_table(averted_rate_space_df)
aggregated_results_df

In [ ]:
aggregated_results_df = aggregated_results_df.reset_index()

In [ ]:
@interact()
def plot_dalys_averted_rate_space(duration=[365.25, 730.50],
                       cgf_permanent=[False, True],
                       iron_permanent=[False, True],
                       iron_mean=[0.895, 4.475, 8.950],
                       measure = aggregated_results_df.measure.unique(),
                       include_other_causes=False,
                      ):
    
    df = aggregated_results_df
    
    data = df.loc[(df.duration == duration)
                  & (df.child_stunting_permanent == cgf_permanent)
                  & (df.child_wasting_permanent == cgf_permanent)
                  & (df.iron_deficiency_permanent == iron_permanent)
                  & (df.iron_deficiency_mean == iron_mean)
                  & (df.measure == measure)]
    
    plt.figure(figsize=(12, 8))
    
    # 'other_causes' value is much higher - can omit by indexing with [:-1]
    displayed_causes = cause_names if include_other_causes else cause_names[:-1]
    for cause in displayed_causes:
        data_sub = data.loc[data.cause == cause]
        
        xx = data_sub['coverage']
        mean = data_sub[('averted', 'mean')]
        lb = data_sub[('averted', '2.5%')]
        ub = data_sub[('averted', '97.5%')]
        
        plt.plot(xx, mean, '-o', label=cause)
        plt.fill_between(xx, lb, ub, alpha=0.1)
    
    plt.title('Nigeria')
    plt.xlabel('Program Coverage (%)')
    plt.ylabel(f'{measure.upper()} Averted (per 100,000 PY)')
    plt.legend(loc=(1.05, .05))
    plt.grid()